In [ ]:
pip install tensorflow tensorflow-datasets


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Settings
IMG_SIZE = 150
BATCH_SIZE = 32
EPOCHS = 3

# Load a smaller subset of the dataset for faster training
(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:10%]', 'train[10%:12%]'],  # Use only 10% for training, 2% for validation
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Preprocessing function: resize and normalize
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

# Prepare datasets
ds_train = ds_train.map(preprocess).shuffle(500).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Build the CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

# Compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model (short run)
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=EPOCHS,
    steps_per_epoch=20,       # Limit steps for quicker training
    validation_steps=5
)

# Save the model
model.save('cats_vs_dogs_small_cnn.h5')


Epoch 1/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.5106 - loss: 0.9006 - val_accuracy: 0.5000 - val_loss: 0.6930
Epoch 2/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.4512 - loss: 0.7031 - val_accuracy: 0.5000 - val_loss: 0.6918
Epoch 3/3
20/20 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.5417 - loss: 0.6916 - val_accuracy: 0.5000 - val_loss: 0.6902
